In [1]:
from bs4 import BeautifulSoup as bs
import urllib
import requests
import pandas as pd
import re
import time

In [4]:
data = pd.read_csv("./Amap_info.csv")
# 删除有地址坐标但没有小区名称的数据
data = data[data['高德名称'].notnull()]
#删除名称中含括号内容的数据，否则在链家网搜获效率很低
data['高德名称'] = data['高德名称'].str.replace(r'\(.*?\)','')
data.drop(data.columns[0], axis=1, inplace=True)
data = data.reset_index(drop = True)
data

,高德ID,高德名称,POI类型,坐标,距离,地址
0,B0FFF3BZ0R,苏宁荣悦怡庭,120302,"121.488675,30.919733",256,解放东路299弄
1,B00155L48N,庙泾新村,120302,"121.490217,30.917393",374,南奉公路与望园南路交叉口东南150米
2,B00156ZH9S,绿地未来中心,120302,"121.489588,30.91577",418,奉贤南桥望园南路1588弄1号
3,B0FFF75YUY,馨雅名筑,120302,"121.482166,30.917704",419,解放东路599弄1-17号
4,B0FFG1OG38,半岛君望,120302,"121.483432,30.915684",420,南奉公路8505号南方国际广场1层
...,...,...,...,...,...,...
155,B00156DGIP,中心公馆,120302,"121.457963,30.909926",2882,江海路388弄8号
156,B00155Q8SM,同盛南桥花园,120302,"121.458177,30.927783",2896,陈桥路299号(横泾港东侧、航南公路南侧)
157,B0FFKBCKM8,光语著,120302,"121.505180,30.897792",2908,南桥镇红旗港路与光宾路交叉口
158,B00155LPOQ,九华新园,120302,"121.460173,30.931629",2911,南桥镇国顺路369弄


In [6]:
# Select a random agent IP adress, and dim the headers
proxy = {'https': '102.164.212.54:8080'}
headers = {'User-Agent':
          "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36 Edg/84.0.522.61"}

In [8]:
new_data = pd.DataFrame(columns = ['高德ID','链家ID','高德名称','链家名称'])
# 在链家网找小区页面，根据高德提供的小区名称，爬取小区对应在链家网的名称以及内部编号
for i in range(len(data)):
    # 连接搜索页获得源码
    quote_name = urllib.parse.quote(data['名称'][i])                                #不支持中英文url地址，因此需要转换
    url_search = 'https://sh.lianjia.com/xiaoqu/fengxian/rs{}'.format(quote_name)
    response_search = requests.get(url_search, headers = headers)
    soup_search = bs(response_search.text, 'html.parser')
    # 查找搜索结果的数量
    search_result_txt = soup_search.find(class_ = "total fl").get_text()
    search_result_num = int(re.findall(r'\d+',search_result_txt)[0])
    # 根据不同的搜索结果分别处理：    
    ## 假如根据高德的小区名称搜索得到结果
    if search_result_num != 0:
        info_content = soup_search.find(class_ = 'listContent').find_all('li')
        for j in range(search_result_num):
            id_lianjia = info_content[j]['data-housecode']
            name_lianjia = info_content[j].find(class_ = 'title').find('a').get_text()                        
            info_dict = {'高德ID' : data['ID'][i],
                         '链家ID' : id_lianjia,
                         '高德名称' : data['名称'][i],
                         '链家名称' : name_lianjia }
            new_data = new_data.append(info_dict, ignore_index=True)
    else:
        continue
    time.sleep(1)

In [9]:
new_data

,高德ID,链家ID,高德名称,链家名称
0,B0FFF3BZ0R,5011102206884,苏宁荣悦怡庭,苏宁荣悦
1,B00155L48N,5011000019384,庙泾新村,庙泾新村
2,B00156ZH9S,5011102206699,绿地未来中心,绿地未来中心
3,B0FFF75YUY,5011063715958,馨雅名筑,馨雅名筑
4,B0FFG1OG38,5011000019801,半岛君望,半岛君望
...,...,...,...,...
159,B00155Q8SM,5011000004160,同盛南桥花园,同盛南桥花园
160,B0FFKBCKM8,5020036484143511,光语著,光语著
161,B00155LPOQ,5011000003475,九华新园,九华新园
162,B00155N813,5011000011571,四季花城,百合苑(公寓)


In [10]:
new_data = new_data.drop_duplicates('链家ID').reset_index(drop=True)
new_data

,高德ID,链家ID,高德名称,链家名称
0,B0FFF3BZ0R,5011102206884,苏宁荣悦怡庭,苏宁荣悦
1,B00155L48N,5011000019384,庙泾新村,庙泾新村
2,B00156ZH9S,5011102206699,绿地未来中心,绿地未来中心
3,B0FFF75YUY,5011063715958,馨雅名筑,馨雅名筑
4,B0FFG1OG38,5011000019801,半岛君望,半岛君望
...,...,...,...,...
101,B00155Q8SM,5011000004160,同盛南桥花园,同盛南桥花园
102,B0FFKBCKM8,5020036484143511,光语著,光语著
103,B00155LPOQ,5011000003475,九华新园,九华新园
104,B00155N813,5011000011571,四季花城,百合苑(公寓)


In [13]:
new_data.to_csv("./Lianjia_info_partial.csv", encoding='utf_8_sig')